# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [2]:
conda install -c conda-forge gmaps

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [27]:
# Set the file path to import the WeatherPy_database.csv file
# city_data_df = pd.read_csv("../Weather_Data/WeatherPy_Database.csv")
# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("./Weather_Database/WeatherPy_Database.csv")
city_data_df.head()
# Display sample data


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kirando,TZ,-7.4167,30.6000,83.26,38,28,8.68,scattered clouds
1,1,Crib Point,AU,-38.3500,145.2000,58.24,94,99,12.55,light rain
2,2,Rafaela,AR,-31.2503,-61.4867,60.98,40,58,3.00,broken clouds
3,3,Saint-Joseph,RE,-21.3667,55.6167,78.10,58,1,12.41,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,69.62,66,2,15.19,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [28]:
# Prompt the user to enter the minimum temperature criteria
min_temp = min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [29]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
# Display sample data
preferred_cities_df.isnull().sum()
preferred_cities_df.head(10)                    


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kirando,TZ,-7.4167,30.6000,83.26,38,28,8.68,scattered clouds
3,3,Saint-Joseph,RE,-21.3667,55.6167,78.10,58,1,12.41,clear sky
6,6,Salalah,OM,17.0151,54.0924,83.70,59,42,10.04,scattered clouds
7,7,Caravelas,BR,-17.7125,-39.2481,75.58,67,57,7.16,broken clouds
8,8,Kampot,KH,10.6167,104.1833,86.05,74,97,6.82,overcast clouds
15,15,Vondrozo,MG,-22.8167,47.2833,79.16,41,20,8.32,few clouds
17,17,East London,ZA,-33.0153,27.9116,87.15,45,100,11.56,overcast clouds
21,21,Port Hedland,AU,-20.3167,118.5667,89.65,11,45,10.36,scattered clouds
23,23,Khandbari,NP,27.3747,87.2039,78.69,67,5,5.26,clear sky
26,26,Hilo,US,19.7297,-155.0900,80.28,92,20,5.75,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [30]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()
print(len(clean_travel_cities))
# Display sample data
clean_travel_cities.head()

240


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kirando,TZ,-7.4167,30.6000,83.26,38,28,8.68,scattered clouds
3,3,Saint-Joseph,RE,-21.3667,55.6167,78.10,58,1,12.41,clear sky
6,6,Salalah,OM,17.0151,54.0924,83.70,59,42,10.04,scattered clouds
7,7,Caravelas,BR,-17.7125,-39.2481,75.58,67,57,7.16,broken clouds
8,8,Kampot,KH,10.6167,104.1833,86.05,74,97,6.82,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [31]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng
0,Kirando,TZ,83.26,scattered clouds,-7.4167,30.6000
3,Saint-Joseph,RE,78.10,clear sky,-21.3667,55.6167
6,Salalah,OM,83.70,scattered clouds,17.0151,54.0924
7,Caravelas,BR,75.58,broken clouds,-17.7125,-39.2481
8,Kampot,KH,86.05,overcast clouds,10.6167,104.1833
15,Vondrozo,MG,79.16,few clouds,-22.8167,47.2833
17,East London,ZA,87.15,overcast clouds,-33.0153,27.9116
21,Port Hedland,AU,89.65,scattered clouds,-20.3167,118.5667
23,Khandbari,NP,78.69,clear sky,27.3747,87.2039
26,Hilo,US,80.28,few clouds,19.7297,-155.0900


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [22]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kirando,TZ,83.26,scattered clouds,-7.4167,30.6000,
3,Saint-Joseph,RE,78.10,clear sky,-21.3667,55.6167,
6,Salalah,OM,83.70,scattered clouds,17.0151,54.0924,
7,Caravelas,BR,75.58,broken clouds,-17.7125,-39.2481,
8,Kampot,KH,86.05,overcast clouds,10.6167,104.1833,
15,Vondrozo,MG,79.16,few clouds,-22.8167,47.2833,
17,East London,ZA,87.15,overcast clouds,-33.0153,27.9116,
21,Port Hedland,AU,89.65,scattered clouds,-20.3167,118.5667,
23,Khandbari,NP,78.69,clear sky,27.3747,87.2039,
26,Hilo,US,80.28,few clouds,19.7297,-155.0900,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [32]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [35]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kirando,TZ,83.26,scattered clouds,-7.4167,30.6000,"Lake Shore Lodge, Tanganyika"
3,Saint-Joseph,RE,78.10,clear sky,-21.3667,55.6167,"""Plantation Bed and Breakfast"
6,Salalah,OM,83.70,scattered clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
7,Caravelas,BR,75.58,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
8,Kampot,KH,86.05,overcast clouds,10.6167,104.1833,Two Moons Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [38]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kirando,TZ,83.26,scattered clouds,-7.4167,30.6000,"Lake Shore Lodge, Tanganyika"
3,Saint-Joseph,RE,78.10,clear sky,-21.3667,55.6167,"""Plantation Bed and Breakfast"
6,Salalah,OM,83.70,scattered clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
7,Caravelas,BR,75.58,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
8,Kampot,KH,86.05,overcast clouds,10.6167,104.1833,Two Moons Hotel
17,East London,ZA,87.15,overcast clouds,-33.0153,27.9116,Tu Casa
21,Port Hedland,AU,89.65,scattered clouds,-20.3167,118.5667,The Esplanade Hotel
23,Khandbari,NP,78.69,clear sky,27.3747,87.2039,Sainik Hotel And Lodge
26,Hilo,US,80.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Wanning,CN,80.19,broken clouds,18.8003,110.3967,Paradise Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [39]:
# Set the file name.
output_data_file = "WeatherPy_Vacation.csv"
# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [40]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [41]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]
# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kirando,TZ,83.26,scattered clouds,-7.4167,30.6000,"Lake Shore Lodge, Tanganyika"
3,Saint-Joseph,RE,78.10,clear sky,-21.3667,55.6167,"""Plantation Bed and Breakfast"
6,Salalah,OM,83.70,scattered clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
7,Caravelas,BR,75.58,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
8,Kampot,KH,86.05,overcast clouds,10.6167,104.1833,Two Moons Hotel


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [1]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# Display the figure containing the map
fig

NameError: name 'gmaps' is not defined